In [1]:
# CELL #1
import matplotlib
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from matplotlib.widgets import Slider, Button, RadioButtons
import matplotlib.gridspec as gridspec

%matplotlib tk

import tkinter as tk
from tkinter import filedialog

import matplotlib.cm as cm
from matplotlib import gridspec
from scipy import signal

import numpy as np
import os
#import shutil
import cv2


import csv


Autosaving every 180 seconds


In [2]:
# CELL #2

def clean_data(event):
    global tracesx, tracesy

    ids = np.arange(4)
    for id_ in ids:
        for k in range(tracesx.shape[0]):
            if np.isnan(tracesx[k,id_])!=np.isnan(tracesy[k,id_]):
                print (k, tracesx[k,id_], tracesy[k,id_])
                tracesx[k,id_]= tracesy[k,id_] =np.nan
                print ('')
                
    print ("DONE matching y and x NANS")
    
    # run delete values on the boundaries
    for id_ in ids:
        idx = np.where(tracesx[:,id_]==0)
        tracesx[idx,id_]=np.nan
        
        idx = np.where(tracesx[:,id_]==1279)
        tracesx[idx,id_]=np.nan
        
        idx = np.where(tracesy[:,id_]==0)
        tracesy[idx,id_]=np.nan

        idx = np.where(tracesy[:,id_]==1023)
        tracesy[idx,id_]=np.nan


    print ("DONe deleting traces on boundaries")

    # delete short segments 
    del_short_segments(0)

    print ("DONE Deleting short segments")
    
    update(current_index[0])
    
def del_singles(event):
    global tracesx, tracesy
    
    # check to see in any sequence of 3 if the middle si the only real value
    ids = np.arange(4)
    for id_ in ids:
        for k in range(0,tracesx.shape[0]-2,1):
            if np.all(np.isnan(tracesx[k:k+3,id_])==np.array([True, 
                                                 False,
                                                 True])):
                tracesx[k:k+3,id_]=np.nan
                tracesy[k:k+3,id_]=np.nan
    print ("DONE DELEting singles")   
    update(current_index[0])
    
    
def del_short_segments(event):
    global tracesx, tracesy
    
    # check to see in any sequence of 3 if the middle si the only real value
    ids = np.arange(4)
    for id_ in ids:
        for k in range(0,tracesx.shape[0]-4,1):
            flag_ = False
            if np.all(np.isnan(tracesx[k:k+5,id_])==np.array([True,
                                                              False,False,False,
                                                              True])):
                tracesx[k:k+5,id_]=np.nan
                tracesy[k:k+5,id_]=np.nan
                flag_ = True
                
            elif np.all(np.isnan(tracesx[k:k+4,id_])==np.array([True,
                                                              False,False,
                                                              True])):
                tracesx[k:k+4,id_]=np.nan
                tracesy[k:k+4,id_]=np.nan
                flag_ = True
                
            elif np.all(np.isnan(tracesx[k:k+3,id_])==np.array([True,
                                                              False,
                                                              True])):
                tracesx[k:k+3,id_]=np.nan
                tracesy[k:k+3,id_]=np.nan 
                flag_ = True
                
            if flag_:
                print ("deleted small segs", id_, k)
    print ("DONE DELEting triples")   
    update(current_index[0])

def find_continous_snippet(current_index, tracesx, id_):
    end=n_frames-1
    for k in range(current_index[0],n_frames,1):
        if np.isnan(tracesx[k,id_]):
            end = k
            break

    # search backward
    start=0
    for k in range(current_index[0],0,-1):
        if np.isnan(tracesx[k,id_]):
            start = k+1
            break

    return end, start

def del_segment(event):
    global tracesx, tracesy, autoadvance
    
    # search active trace snippet
    #end2, start2 = find_continous_snippet(current_index, tracesx, switch_id)
    end1, start1 = find_continous_snippet(current_index, tracesx, selected_animal[0])

    print ("Selected animal id: ", selected_animal, ", start/end: ", start1, end1)

    # replace switch id info
    tracesx[start1:end1, selected_animal[0]] = np.nan
    tracesy[start1:end1, selected_animal[0]] = np.nan
  
    # move to the next snippet
    if autoadvance:
        print ("audo advance: ", autoadvance)
        next_segment(0)
        
    update(current_index[0])
    
def del_segment2(event):
    global tracesx, tracesy, autoadvance, state_switch, state_switch_selected_id

        
    # search active trace snippet
    # find which switch
    #end2, start2 = find_continous_snippet(current_index, tracesx, switch_id)
    print ("del button 2, animal_id set to state_switch_selected_id", state_switch_selected_id)

    
    end1, start1 = find_continous_snippet(current_index, tracesx, state_switch_selected_id)

    print ("Selected animal id: ", state_switch_selected_id, ", start/end: ", start1, end1)

    # replace switch id info
    tracesx[start1:end1, state_switch_selected_id] = np.nan
    tracesy[start1:end1, state_switch_selected_id] = np.nan
  
    # move to the next snippet
    if autoadvance:
        print ("audo advance: ", autoadvance)
        next_segment(0)
        
    update(current_index[0])
    
# velocity based error correcstion
# principle: the Delta frame jump can't be more than 50% of the previous delta jump
def disconnect_jumps_velocity(start=None, end=None):
    global tracesx, tracesy
    

    if start is None:
        start = 0
        end = tracesx.shape[0]-1
    
    max_acceleration = 5
    min_velocity = 30
    max_velocity = 200
    for k in range(tracesx.shape[1]):
        # precompute all jumps
        DIST_x = tracesx[:-1,k]-tracesx[1:,k]
        DIST_y = tracesy[:-1,k]-tracesy[1:,k]
        velocity = np.sqrt(DIST_x**2+DIST_y**2)
        print ("velocity: ", velocity.shape, velocity)
        
        # remove L2 errors as max velocity errros
        idx = np.where(velocity>max_velocity)[0]
        for id_ in idx:
            tracesx[id_:id_+2,k] = np.nan
            tracesy[id_:id_+2,k] = np.nan        
        
        # next check places where velocity is over minimum value:
        idx1 = np.where(velocity>=min_velocity)[0]
    
        # and remove those that are way faster than previous
        idx2 = np.where(velocity[idx1]>(velocity[idx1+1]*max_acceleration))[0]
        #print ("idx: ", idx)
        for id_ in idx2:
            tracesx[idx1[id_]:idx1[id_]+2,k] = np.nan
            tracesy[idx1[id_]:idx1[id_]+2,k] = np.nan

    print ("DONE disconnecting JUMPS")
    
    return tracesx, tracesy


# def disconnect_jumps_L1(start=None, end=None):
#     ''' Removes only Linf errors occuring along one directoin 
    
#     '''
    
#     global tracesx, tracesy
    
#     #tracesx_ip=tracesx.copy()
#     #tracesy_ip=tracesy.copy()
#     if start is None:
#         start = 0
#         end = tracesx.shape[0]-1
    
#     max_jump_L2 = 300
#     max_jump_L1 = 50
#     min_jump_L1 = 2

#     # Fix bad L2 errors
#     for k in range(tracesx.shape[1]):
#         for p in range(start,end,1):
#             distx = tracesx[p,k]-tracesx[p+1,k]
#             disty = tracesy[p,k]-tracesy[p+1,k]
#             dist = np.sqrt(distx**2+disty**2)
#             if dist>max_jump_L2:
#                 #print (snippets[p,0],snippets[p,1])
#                 tracesx[p:p+2,k]= np.nan
#                 tracesy[p:p+2,k]= np.nan
#             elif distx>max_jump_L1 and disty<min_jump_L1:
#                 #print (snippets[p,0],snippets[p,1])
#                 tracesx[p:p+2,k]= np.nan
#                 tracesy[p:p+2,k]= np.nan

#             elif disty>max_jump_L1 and distx<min_jump_L1:
#                 #print (snippets[p,0],snippets[p,1])
#                 tracesx[p:p+2,k]= np.nan
#                 tracesy[p:p+2,k]= np.nan
#                 #print ("found large jump: ", p/25.)

    
#     return tracesx, tracesy

       
def find_all_continous_snippets(tracesx_temp, tracesy_temp):
        
    idx = np.where(~np.isnan(tracesx_temp))[0]

    # make snippets
    snippets=[]
    locs = []
    start = idx[0]
    #for p in range(1,idx.shape[0],1):
    p=1
    ctr=0
    while True:
        if idx[p]-idx[p-1]>1:
            end = idx[p-1]
            snippets.append([start,end])
            start = idx[p]   

            # find locations of start and end
            locs.append([])
            locs[ctr].append([tracesx_temp[start], tracesy_temp[start]])
            locs[ctr].append([tracesx_temp[end], tracesy_temp[end]])

            # bump 2 over
            p+=2
            ctr+=1
        else:
            p+=1

        if p>=idx.shape[0]:
            break

    snippets = np.array(snippets)
    locs = np.array(locs)

    return snippets, locs


# TODO Add time threshold also; 
def interpolate_traces(dist_threshold=10., time_threshold=10):
    
    global tracesx, tracesy
    
    for k in range(tracesx.shape[1]):

        snippets, locs = find_all_continous_snippets(tracesx[:,k], tracesy[:,k])

        # fill in space between snippets if euclidean distance < threshold
        #threshold = 10.
        # AND IF NOT TOO FAR APART; 
        for p in range(1, snippets.shape[0]-1):
            # Skip if interpolate across too long a sgement
            if (snippets[p,0] - snippets[p-1,1])>time_threshold:
                continue
            dist = np.linalg.norm(locs[p,0]-locs[p-1,1])
            
            #print ("Dist; ", dist, locs[p,1],locs[p+1,0])
            if dist<dist_threshold:
                #print ("interpoalting snippets: ", k, snippets[p,0],snippets[p,1])
                tracesx[snippets[p-1,1]:snippets[p,0]+1,k]= tracesx[snippets[p-1,1],k]
                tracesy[snippets[p-1,1]:snippets[p,0]+1,k]= tracesy[snippets[p-1,1],k]
            #else:
            #    print (dist, "not merging")

    # 
    print ("DELETING JUMPS FUCNTION")
    #tracesx, tracesy = disconnect_jumps(tracesx,tracesx)

    update(current_index[0])
    #fig.canvas.draw_idle()
    
    return tracesx, tracesy


# update the slider
def update(val):
    global tracesx, tracesy, sizes, img, original_vid, axes_animals, ax_tracesx, ax_tracesy, fig

    #amp = samp.val
    if int(sfreq.val)!=current_index[0]:
        index = int(sfreq.val)
        current_index[0] = index
    else:
        index = current_index[0]
    print ("current indx updated:", current_index)
    
    # grab frame and update it
    original_vid.set(cv2.CAP_PROP_POS_FRAMES, index)
    ret, frame = original_vid.read()
    img.set_data(frame)

    # update scatter plots
    for k in range(4):
        x_loc = tracesx[index,k]
        y_loc = tracesy[index,k]
        
        # If no location inferred, freeze and turn off labels
        if np.isnan(x_loc) or np.isnan(y_loc):
            axes_animals[k].set_alpha(0.0)
            continue
        else:
            axes_animals[k].set_alpha(1.0)
            axes_animals[k].set_sizes([sizes[k]])
            axes_animals[k].set_offsets(np.c_[x_loc,y_loc])
    
    # update trace plots
    ax_tracesx.clear()
    ax_tracesx.set_ylim(0,1280)

    ax_tracesy.clear()
    ax_tracesy.set_ylim(0,1024)
    for k in range(4):
        ax_tracesx.plot(t+current_index[0]/25., tracesx[index:index+init_length,k],
                        linewidth=sizes[k]//50,
                       c=clrs[k])
        #ax1[k].set_linewidth(sizes[k]//50)
        #ax2[k].set_ydata(tracesy[index:index+init_length,k])    
        #ax2[k].set_linewidth(sizes[k]//50)
        ax_tracesy.plot(t+current_index[0]/25., tracesy[index:index+init_length,k], 
                        linewidth=sizes[k]//50,
                       c=clrs[k])
        
    #print ("t value: ", t)
    #print ("second t: ", t+current_index[0]/25.)
    fig.canvas.draw_idle()
    
def advance_function(label):
    global autoadvance
    print ("label: ", label)
    if label == "ON":
        autoadvance=1
    else:
        autoadvance=0
        
    print ("Auto advance: ", autoadvance)
 
def multi_animal_skip(label):
    global multianimal
    print ("label: ", label)
    if label == "ON":
        multianimal=1
    else:
        multianimal=0
        
    print ("multianimal: ", multianimal)    


def animal_select(label):
    #l.set_color(label)
    #fig.canvas.draw_idle()
    
    sizes[:] = 50
    for k in range(4):
        if labels[k] == label:
            break
    sizes[k]=250
    selected_animal[0]=k
    update(current_index[0])

def switch_all_screen(event):    

    global tracesx, tracesy
    print ('')
    print ('')
    print ("SWITCH ALL TRACES ON SCREEN BETWEEN 2 ANIMALS")

    label = animal_switch.value_selected
    print ("label: ", label)
    for k in range(4):
        if labels[k] == label:
            break
    switch_id = k    
    
    print ("curent animal: ", selected_animal[0], " switch animal = ", k )
    
    start = current_index[0]
    end = current_index[0]*10*25-1  # swap all 10 seconds visible on screen
    
    tempx = tracesx.copy()
    tempy = tracesy.copy()
    
    # replace switch id info
    tracesx[start:end, switch_id] = tempx[start:end, selected_animal[0]].copy()
    tracesy[start:end, switch_id] = tempy[start:end, selected_animal[0]].copy()
    
    # replace currental animal info
    tracesx[start:end, selected_animal[0]] = tempx[start:end, switch_id].copy()
    tracesy[start:end, selected_animal[0]] = tempy[start:end, switch_id].copy()
    
    # update plots
    update(current_index[0])   
    
    
def animal_swap(label):
    #l.set_color(label)
    global tracesx, tracesy
    print ('')
    print ('')
    print ("SWITCH ALL TRACES BETWEEN 2 ANIMALS FUNCTION")

    label = animal_switch.value_selected
    print ("label: ", label)
    for k in range(4):
        if labels[k] == label:
            break
    switch_id = k    
    
    print ("curent animal: ", selected_animal[0], " switch animal = ", k )
    
    tempx = tracesx.copy()
    tempy = tracesy.copy()
    
    # replace switch id info
    tracesx[:, switch_id] = tempx[:, selected_animal[0]].copy()
    tracesy[:, switch_id] = tempy[:, selected_animal[0]].copy()
    
    # replace currental animal info
    tracesx[:, selected_animal[0]] = tempx[:, switch_id].copy()
    tracesy[:, selected_animal[0]] = tempy[:, switch_id].copy()
    
    # update plots
    update(current_index[0])   
 
def animal_switch_function(label, mouse_swap=False, id1=None, id2=None):
    #l.set_color(label)
    
    global tracesx, tracesy, autoadvance
    print ('')
    print ('')
    print ("SWITCH FUNCTION")

    if mouse_swap==False:
        for k in range(4):
            if labels[k] == label:
                break
        #switch_id = k
        id2 = k
        id1 = selected_animal[0]
#     else:
#         id1 = id1
#         id1 = id2
    print ("incoming ids; ", id1, id2)
        
    # look for snippets
    def find_continous_snippet(current_index, tracesx, id_):
        end=n_frames-1
        for k in range(current_index[0],n_frames,1):
            if np.isnan(tracesx[k,id_]):
                end = k
                break

        # search backward
        start=0
        for k in range(current_index[0],0,-1):
            if np.isnan(tracesx[k,id_]):
                start = k+1
                break
    
        return end, start
    
    #print ("switching to : ", labels[switch_id])
    print ("switching to : ", labels[id2])
    print ("current index inside switch:", current_index)
    
    #end2, start2 = find_continous_snippet(current_index, tracesx, switch_id)
    end2, start2 = find_continous_snippet(current_index, tracesx, id2)
            
    #print ("Switch animal id: ", switch_id, ", start/end: ", start2, end2)
    print ("Switch animal id: ", id2, ", start/end: ", start2, end2)
    
    #end1, start1 = find_continous_snippet(current_index, tracesx, selected_animal[0])
    end1, start1 = find_continous_snippet(current_index, tracesx, id1)
    
    #print ("selected animal id: ", selected_animal[0], ", start/end: ", start1, end1)
    print ("selected animal id: ", id1, ", start/end: ", start1, end1)
        
    
    if end2>end1:
        end1=end2
    
    # DATA SWITCH HAS TO BE SAFE FOR ALL DOWNSTREAM 
    # SO NEED TO CHECK THE ID THAT"S BEING SWAPPED TO MAKE SURE IT DOESN"T GET DELETED;
    # switch the data
    tempx = tracesx.copy()
    tempy = tracesy.copy()
    
#     print ("Target switch tracesx: ", tempx[start1:end1, switch_id].copy())
#     print ("Selected animal tracesx: ", tempx[start1:end1, selected_animal[0]].copy())

#     buff = 2
#     print ("Switch ANIMAL TRACES preswitch; animal: ", switch_id,
#           tracesx[start1-buff:end1+buff, switch_id],
#           tracesy[start1-buff:end1+buff, switch_id])
        

    # replace switch id info
    #tracesx[start1:end1, switch_id] = tempx[start1:end1, selected_animal[0]].copy()
    #tracesy[start1:end1, switch_id] = tempy[start1:end1, selected_animal[0]].copy()
    tracesx[start1:end1, id2] = tempx[start1:end1, id1].copy()
    tracesy[start1:end1, id2] = tempy[start1:end1, id1].copy()
    
#     print ("Switch ANIMAL TRACES postswitch; animal: ", switch_id, 
#           tracesx[start1-buff:end1+buff, switch_id],
#           tracesy[start1-buff:end1+buff, switch_id])
           
#     print ("Selected ANIMAL TRACES preswitch; animal: ",selected_animal[0],
#           tracesx[start1-buff:end1+buff, selected_animal[0]],
#           tracesy[start1-buff:end1+buff, selected_animal[0]])

    # replace currental animal info
    #tracesx[start1:end1, selected_animal[0]] = tempx[start1:end1, switch_id].copy()
    #tracesy[start1:end1, selected_animal[0]] = tempy[start1:end1, switch_id].copy()
    tracesx[start1:end1, id1] = tempx[start1:end1, id2].copy()
    tracesy[start1:end1, id1] = tempy[start1:end1, id2].copy()
    
#     print ("SelectedANIMAL TRACES postswitch; animal ",selected_animal[0],
#           tracesx[start1-buff:end1+buff, selected_animal[0]],
#           tracesy[start1-buff:end1+buff, selected_animal[0]])
  
    # 
    #print ("DELETING JUMPS FUCNTION")
    #disconnect_jumps()

    # move to the next snippet
    if autoadvance:
        print ("audo advance: ", autoadvance)
        next_segment(0)
    
    # run disconnect jumps on this section of data
    if start1>0:
        disconnect_jumps_velocity(start=start1-1, end=end1)
    else:
        disconnect_jumps_velocity(start=start1, end=end1)
        
    # 
    update(current_index[0])   
    

# search for the next continuous chunk of red
def next_segment(event):

    global multianimal, selected_animal, current_index, sizes
    
    # 
    # find the next snippet for the currently selected animal
    if multianimal==0:
        start_nan = n_frames-1
        for k in range(current_index[0], n_frames, 1):
            if np.isnan(tracesx[k,selected_animal[0]]):
                start_nan = k
                break
        print ("searching... start_nan: ", start_nan)

        start_nan+=1
        start_val = n_frames-1
        for k in range(start_nan, n_frames, 1):
            if np.isnan(tracesx[k,selected_animal[0]])==False:
                start_val=k
                break

        print ("Fast forwarding from: ", current_index[0])
        current_index[0] = start_val
        
        sfreq.val = current_index[0]
        #fig.canvas.draw_idle()

        print ("..... to : ", current_index[0])

        update(current_index[0])

    # Search all snippets for the next valid 
    else:
        print ("NOT IMPLENETED YET")
        pass
        start_val = n_frames-1
        for l in range(4):
            if l==selected_animal[0]:
                continue
            
            # search for first non nan value and animal id
            for k in range(current_index[0], n_frames, 1):
                if np.isnan(tracesx[k,l])==False:
                    if start_val>=k:
                        selected_animal[0] = l
                        start_val = k
                        break
            
                    
        print ("Fast forwarding from: ", current_index[0])
        current_index[0] = start_val
    
   
    
    
    # search for the next continuous chunk of red

    
def forward_function(event):
    
    # 
    current_index[0]+=1
    sfreq.val = current_index[0]

    update(current_index[0])
    #fig.canvas.draw_idle()
    
def backward_function(event):
    
    # 
    current_index[0]-=1
    sfreq.val = current_index[0]

    update(current_index[0])
    #fig.canvas.draw_idle()

        
def disconnect_jumps_function(event):
    
    global tracesx, tracesy
    # 
    print ("DELETING JUMPS FUCNTION")
    disconnect_jumps_velocity()

    update(current_index[0])
    #fig.canvas.draw_idle()
    
    
def previous_segment(event):
    
    # 
    print ("Current esgemtn: ", tracesx[current_index[0]:current_index[0]+5,
                                        selected_animal[0]])
    
    # END OF PREVIOUS NAN BLOCK
    end_nan=0
    for k in range(current_index[0], 0, -1):
        if np.isnan(tracesx[k,selected_animal[0]]):
            end_nan = k
            break
    print ("searching... start_nan: ", end_nan)

    # START OF PREVIOUS NAN BLOCK
    start_nan=0
    for k in range(end_nan, 0, -1):
        if np.isnan(tracesx[k,selected_animal[0]])==False:
            start_nan=k+1
            break

    start_data =0
    for k in range(start_nan-1, 0, -1):
        if np.isnan(tracesx[k,selected_animal[0]]):
            start_data=k+1 # select the very next val
            break
            
    print ("Reversing from: ", current_index[0])
    current_index[0] = start_data
    sfreq.val = current_index[0]

    print ("..... to : ", current_index[0])
    update(current_index[0])
    #fig.canvas.draw_idle()

    
def save(event):
    
    pattern = (('NPZ', '*.npz'),)
    title = 'Npz'
    
    traces_saved = filedialog.asksaveasfilename(filetypes=pattern, title=title)
    
    np.savez(traces_saved, 
             tracesx = tracesx,
             tracesy = tracesy)
    print ("SAVING TRACES")
        
def load_function(event):
    
    global tracesx, tracesy
    
    pattern = (('NPZ', '*.npz'),)
    title = 'Npz'

    traces_saved = filedialog.askopenfilename(filetypes=pattern, title=title)
    
    data = np.load(traces_saved)
    tracesx = data['tracesx']
    tracesy = data['tracesy']
    update(current_index[0])

    print ("loaded traces")

    
def interpolate_function(event):
    global tracesx, tracesy
    
    dist_threshold = 200.
    time_threshold = 25
    tracesx, tracesy = interpolate_traces(dist_threshold,
                                         time_threshold)
    
    update(current_index[0])
    #fig.canvas.draw_idle()
    print ("INTERPOLATING TRACES, THRESHOLD: ", dist_threshold)

    
    
def export_movie(movie):
    clrs = ['blue','red','cyan','green']

    size_vid = np.array([1280,1024])
    
    scaling = 4
    dot_size = 16//scaling
    
    #out_dir = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/'
    fname_out = (out_dir+'video_labeled_exported_fixed.avi')
    fourcc = cv2.VideoWriter_fourcc('M','P','E','G')
    video_out = cv2.VideoWriter(fname_out,fourcc, 25, (size_vid[0]//scaling,size_vid[1]//scaling), True)

    # initialize start and ends of video to be analyzed
    start = 0
    #end = start + 10*25
    end = tracesx.shape[0]
    #end = 3000
    original_vid = cv2.VideoCapture(video_name)
    original_vid.set(cv2.CAP_PROP_POS_FRAMES, start)

    for k in range(start,end,1):    
        if k %2500==0:
            print (k)

        ret, frame = original_vid.read()
        if ret==False:
            break
#         if scaling:
#             frame_cropped = frame.copy()[::4,::4]
#         else:
#             frame_cropped = frame.copy()
        frame = frame[::scaling, ::scaling]

        # Loop over animals
        for p in range(0, tracesx.shape[1], 1):
            x = tracesx[k,p]
            y = tracesy[k,p]
            # plot large bump
            if np.isnan(x) or np.isnan(y):
                continue
            x = int(x)//scaling
            y = int(y)//scaling
            
            frame[y-dot_size:y+dot_size,x-dot_size:x+dot_size]= (np.float32(
                matplotlib.colors.to_rgb(clrs[p]))*255.).astype('uint8')

        video_out.write(frame)

    video_out.release()
    original_vid.release()
    cv2.destroyAllWindows()
    
    

def load_dataset(event):
    
    global state_switch, tracesx, tracesy, video_name, out_dir, original_vid, axes_animals, ax_tracesx, ax_tracesy, n_frames, img, t, init_length, sfreq
    
    
    pattern = (('AVI', '*.avi'),)
    title = 'Load .avi video to review'

    video_name = filedialog.askopenfilename(filetypes=pattern, title=title)
    
    out_dir = os.path.split(video_name)[0]
    
    
    # initialize video box
    original_vid = cv2.VideoCapture(video_name)
    #dlc.original_vid = original_vid


    # initialize video
    loc = 0
    original_vid.set(cv2.CAP_PROP_POS_FRAMES, loc)
    ret, frame = original_vid.read()
    n_frames = int(original_vid.get(cv2.CAP_PROP_FRAME_COUNT))

    axcolor = 'lightgoldenrodyellow'
    axfreq = plt.axes([0.01, 0.01, 0.94, 0.03], facecolor=axcolor)
    
    #sfreq = Slider(axfreq, 'Frame', 0, n_frames, valinit=0, valstep=1, dragging=True)
    sfreq = Slider(axfreq, 'Frame', 0, n_frames, valinit=0, valstep=1, dragging=True)
    sfreq.on_changed(update)

    # initialize scatter plots
    ax_video = plt.subplot(gs[:, 1:4])
    img = ax_video.imshow(frame)
    ax_video.set_xlim(125,1225)
    ax_video.set_ylim(50,800)
    
    state_switch = False
    cid = fig.canvas.mpl_connect('button_press_event', onclick_mouse)

    
    # load traces also
    pattern = (('NPZ', '*.npz'),)
    title = 'Load .npz file containing traces'

    traces_saved = filedialog.askopenfilename(filetypes=pattern, title=title)
    
    # 
    data = np.load(traces_saved)
    tracesx = data['tracesx']
    tracesy = data['tracesy']
    
    # initialize traces:
    axes_animals=[]
    for k in range(4):
        x_loc = tracesx[0,k]
        y_loc = tracesy[0,k]
        if np.isnan(x_loc) or np.isnan(y_loc):
            x_loc=100
            y_loc=100
        axes_animals.append(ax_video.scatter(x_loc, y_loc,c=clrs[k]))

    # plot traces
    ax_tracesx = plt.subplot(gs[:2, :1])
    ax_tracesx.set_title("DLC-x")
    init_length = 10*25
    t = np.arange(init_length)/25.
    ax1 = []
    for k in range(4):
        temp, = ax_tracesx.plot(t,tracesx[:init_length,k],c=clrs[k])
        ax1.append(temp)

    ax_tracesx.set_xticks([])
    ax_tracesx.set_ylim(0,1280)

    # plot y traces
    ax_tracesy = plt.subplot(gs[2:, :1])
    ax_tracesy.set_title("DLC-y")
    ax2 = []
    for k in range(4):
        temp, = ax_tracesy.plot(t,tracesy[:init_length,k],c=clrs[k])
        ax2.append(temp)

    ax_tracesy.set_xlabel("Time (sec)")
    ax_tracesy.set_ylim(0,1024)

    # initialize scatter plot

    #video_name = os.path.join(out_dir, '2020-3-9_08_18_49_128168_compressed_snippet.avi')

    update(current_index[0])

    print ("loaded video")
        
def onclick_mouse(event):
    import scipy
    
    global state_switch, state_switch_selected_id
    print('%s click: button=%d, x=%d, y=%d, xdata=%f, ydata=%f' %
          ('double' if event.dblclick else 'single', event.button,
           event.x, event.y, event.xdata, event.ydata))
    
    y = event.ydata
    x = event.xdata

    #if event.button==3:
    if True:
        if y>55 and y<95:
            if x>0 and x<400: 
                print ("color box")
                if x>0 and x<100:
                    print ("red")
                    switch_id = 0
                if x>100 and x<200:
                    print ("blue")
                    switch_id = 1

                if x>200 and x<300:
                    print ("cyan")
                    switch_id = 2

                if x>300 and x<400:
                    print ("green")
                    switch_id = 3

            # run the animal siwthc
            print ("Doing mouse-based switch")
            #print (state_switch_selected_id, min_arg)
            animal_switch_function(label=0, 
                                   mouse_swap=True, 
                                   id1=state_switch_selected_id, 
                                   id2=switch_id)    

            # reset
            state_switch_selected_id = None

            update(current_index[0])
        # check to see if in video window
        #  xdata=128.882826, ydata=53.432134
        #  xdata=129.919147, ydata=794.401954
        #  xdata=1222.201932, ydata=795.438276
        #  xdata=1222.201932, ydata=55.504777
    
        elif y>55 and y<794 and x>128 and x<1222:
            
        
            print ('selection of mouse')
            loc_mouse = np.array([event.xdata, event.ydata])



            # find nearest point to the event.xdata and event.ydata:
            if int(sfreq.val)!=current_index[0]:
                index = int(sfreq.val)
                current_index[0] = index
            else:
                index = current_index[0]

            # get locations of all 4 animals
            x_locs = tracesx[current_index,:]
            y_locs = tracesy[current_index,:]
            print ("current indx updated:", current_index)
            print ("xlocs: ", x_locs)
            print ('ylocs:', y_locs)
            all_locs = np.zeros((4,2),'float32')
            all_locs[:,0] = x_locs
            all_locs[:,1] = y_locs

            vect = all_locs-loc_mouse
            print ("vect: ", vect)
            print ("Vect: ", vect.shape)
            idx = np.where(np.isnan(vect))
            vect[idx]=1E10
            min_arg = np.nanargmin(scipy.spatial.distance.cdist(vect*0, vect))

            print ("Nearest animal ID: ", min_arg)
            state_switch_selected_id = min_arg
       

In [3]:
# CELL 3 LOAD GUI
global state_switch_selected_id, state_switch, current_index, selected_animal, tracesx, tracesy, traces_saved, n_frames, autoadvance, \
        frame_rate, sfreq, original_vid, axes_animals, ax_tracesx, ax_tracesy, fig

# set plotting figure
fig = plt.figure()
gs = gridspec.GridSpec(4, 4)
gs.tight_layout(fig)

clrs = ['red','blue','cyan','green']
sizes = np.ones(4,'int32')*50
sizes[0] = 250

# auto advance default setting
autoadvance = 0

# multianimal setting: cycle through segments for all animals, not just specific animal
multianimal = 0

# start with default animal id
state_switch_selected_id = None

# save current frame of video
current_index = []
current_index.append(0)
#dlc.current_index=current_index

# save selected animal radio button 
selected_animal = []
selected_animal.append(0)

# initialize start and ends of video to be analyzed
frame_rate = 25
# ##################### BUTTONS #####################

# Auto advance SELECTOR
axcolor = 'lightgoldenrodyellow'
labels = ["OFF", "ON"]
rax0 = plt.axes([0.01, 0.90, 0.04, 0.05], facecolor=axcolor)
advance_buttons = RadioButtons(rax0, (labels[0],labels[1]), active=0)
rax0.set_title('Auto\nadvance')
advance_buttons.on_clicked(advance_function)
for circle in advance_buttons.circles: # adjust radius here. The default is 0.05
    circle.set_radius(0.1)

# #labels = ["OFF", "ON"]
# rax00 = plt.axes([0.06, 0.90, 0.04, 0.05], facecolor=axcolor)
# multi_animal = RadioButtons(rax00, (labels[0],labels[1]), active=0)
# rax00.set_title('Multi\nAnimal')
# multi_animal.on_clicked(multi_animal_skip)
# for circle in multi_animal.circles: # adjust radius here. The default is 0.05
#     circle.set_radius(0.1)


#axprev = plt.axes([0.7, 0.05, 0.1, 0.075])
axprev = plt.axes([0.15, 0.90, 0.07, 0.10])
bprev = Button(axprev, 'Previous\nSegment')
bprev.on_clicked(previous_segment)
#bprev = Button(axprev, 'Previous')
#bprev.on_clicked(callback.prev)

# NEXT BUTTON
axnext = plt.axes([0.24, 0.90, 0.07, 0.10])
bnext = Button(axnext, 'Next\nSegment')
bnext.on_clicked(next_segment)
#bprev = Button(axprev, 'Previous')
#bprev.on_clicked(callback.prev)


# Del BUTTON2
axdel2 = plt.axes([0.33, 0.90, 0.07, 0.10])
bdel2 = Button(axdel2, 'Del Highlighted\nSegment')
bdel2.on_clicked(del_segment2)
#bprev = Button(axprev, 'Previous')
#bprev.on_clicked(callback.prev)

# # color bar
rax001 = plt.axes([0.34, 0.90, 0.7, 0.22], facecolor=axcolor)
rax001.set_xticks([])
rax001.set_yticks([])
img = np.zeros((100,400,3),'float32')


red = [1,0,0]
blue = [0,0,1]
cyan = [0,1,1]
green = [0,1,0]

img[:,:100] = red
img[:,100:200] = blue
img[:,200:300] = cyan
img[:,300:] = green

rax001.imshow(img)



# ANIMAL SELECTOR
labels = ["female (red)", "male (blue)", "pup1 (cyan)", "pup2 (green)"]
rax1 = plt.axes([0.01, 0.80, 0.07, 0.08], facecolor=axcolor)
animal_buttons = RadioButtons(rax1, (labels[0],labels[1],labels[2],labels[3]), active=0)
rax1.set_title('Select')
animal_buttons.on_clicked(animal_select)
for circle in animal_buttons.circles: # adjust radius here. The default is 0.05
    circle.set_radius(0.1)
    
# # ANIMAL SWITCHER
# rax2 = plt.axes([0.01, 0.70, 0.07, 0.08], facecolor=axcolor)
# animal_switch = RadioButtons(rax2, (labels[0],labels[1],labels[2],labels[3]), active=0)
# rax2.set_title('Switch') 
# animal_switch.on_clicked(animal_switch_function)
# for circle in animal_switch.circles: # adjust radius here. The default is 0.05
#     circle.set_radius(0.1)
    


# Del Segment
axdel_seg = plt.axes([0.06, 0.65, 0.04, 0.04])
bdel_seg = Button(axdel_seg, 'Del\nSegment')
bdel_seg.on_clicked(del_segment)



#axprev = plt.axes([0.7, 0.05, 0.1, 0.075])
axforward = plt.axes([0.01, 0.50, 0.04, 0.04])
bforward = Button(axforward, 'Step\nForward')
bforward.on_clicked(forward_function)

# BACK BUTTON
axback = plt.axes([0.06, 0.50, 0.04, 0.04])
bback = Button(axback, 'Step\nBackward')
bback.on_clicked(backward_function)
#bprev = Button(axprev, 'Previous')




# INTERPOLATE
axnans = plt.axes([0.01, 0.40, 0.04, 0.04])
bnans = Button(axnans, 'Clean\nData',  color='0.85', hovercolor='0.95')
bnans.on_clicked(clean_data)
bnans.color='grey'
bnans.hovercolor='green'


# DELETE JUMPS BUTTON
axdel_jumps = plt.axes([0.01, 0.35, 0.04, 0.04])
bdeljumps = Button(axdel_jumps, 'Disconnect\nJumps')
bdeljumps.on_clicked(disconnect_jumps_function)
#bprev = Button(axprev, 'Previous')

# INTERPOLATE
axinterpolate = plt.axes([0.06, 0.35, 0.04, 0.04])
binterpoloate = Button(axinterpolate, 'Join\nTraces')
binterpoloate.on_clicked(interpolate_function)
#bprev = Button(axprev, 'Previous')

   
    
#axprev = plt.axes([0.7, 0.05, 0.1, 0.075])
axswitch_all = plt.axes([0.01, 0.20, 0.04, 0.04])
bswitch_all = Button(axswitch_all, 'Switch\nTraces (10sec)')
bswitch_all.on_clicked(switch_all_screen)

# Export movie
axmovie = plt.axes([0.01, 0.05, 0.04, 0.04])
bmovie = Button(axmovie, 'Export\nMovie')
bmovie.on_clicked(export_movie)
#bprev = Button(axprev, 'Previous')


#axprev = plt.axes([0.7, 0.05, 0.1, 0.075])
axload_dataset = plt.axes([0.06, 0.15, 0.04, 0.04])
bload_dataset = Button(axload_dataset, 'Load\nDataset', color='0.85', hovercolor='0.95')
bload_dataset.color='grey'
bload_dataset.hovercolor='r'

#button = Button(tkWindow, text = 'Submit', bg='blue', fg='white')  

bload_dataset.on_clicked(load_dataset)


#axprev = plt.axes([0.7, 0.05, 0.1, 0.075])
axsave = plt.axes([0.06, 0.05, 0.04, 0.04])
bsave = Button(axsave, 'Save\nTraces')
bsave.on_clicked(save)

#axprev = plt.axes([0.7, 0.05, 0.1, 0.075])
axload = plt.axes([0.11, 0.05, 0.04, 0.04])
bload = Button(axload, 'Load\nTraces')
bload.on_clicked(load_function)


#load_dataset(0)
    

plt.show()

current indx updated: [0]
loaded video
current indx updated: [21413]
single click: button=1, x=433, y=18, xdata=21412.847688, ydata=0.272116
current indx updated: [22188]
current indx updated: [23014]
current indx updated: [23479]
current indx updated: [23996]
current indx updated: [25443]
current indx updated: [29317]
current indx updated: [29627]
single click: button=1, x=881, y=744, xdata=409.725933, ydata=687.660847
selection of mouse
current indx updated: [29627]
xlocs:  [[259.1856  394.27386       nan 309.97223]]
ylocs: [[665.36005 694.65704       nan 584.0288 ]]
vect:  [[-150.54032501  -22.30080104]
 [ -15.45206817    6.99619603]
 [          nan           nan]
 [ -99.75370392 -103.63203883]]
Vect:  (4, 2)
Nearest animal ID:  1
single click: button=1, x=958, y=958, xdata=52.463031, ydata=69.181681
color box
red
Doing mouse-based switch


SWITCH FUNCTION
incoming ids;  1 0
switching to :  female (red)
current index inside switch: [29627]
Switch animal id:  0 , start/end:  29599 29

Traceback (most recent call last):
  File "/home/cat/anaconda3/lib/python3.8/site-packages/matplotlib/cbook/__init__.py", line 196, in process
    func(*args, **kwargs)
  File "<ipython-input-2-48ae7c6b5393>", line 892, in onclick_mouse
    print('%s click: button=%d, x=%d, y=%d, xdata=%f, ydata=%f' %
TypeError: must be real number, not NoneType


current indx updated: [42698]
single click: button=1, x=845, y=28, xdata=42698.087265, ydata=0.608476
current indx updated: [47451]
current indx updated: [47554]
current indx updated: [12837]
single click: button=1, x=267, y=25, xdata=12836.756014, ydata=0.507568
current indx updated: [12888]
current indx updated: [12940]
single click: button=1, x=762, y=690, xdata=286.403683, ydata=631.699490
selection of mouse
current indx updated: [12940]
xlocs:  [[1044.6871  282.8451  980.4411 1078.6749]]
ylocs: [[727.176   631.1668  378.8392  141.13466]]
vect:  [[ 7.58283451e+02  9.54765351e+01]
 [-3.55859026e+00 -5.32681257e-01]
 [ 6.94037418e+02 -2.52860287e+02]
 [ 7.92271244e+02 -4.90564832e+02]]
Vect:  (4, 2)
Nearest animal ID:  1
single click: button=1, x=932, y=934, xdata=40.537519, ydata=80.189845
color box
red
Doing mouse-based switch


SWITCH FUNCTION
incoming ids;  1 0
switching to :  female (red)
current index inside switch: [12940]
Switch animal id:  0 , start/end:  12915 12958
selecte

In [26]:
#data = np.load('/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_01_54_23_358257_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npz')
data = np.load('/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/cat_sep_29_2.8hrs.npz',allow_pickle=True)
tracesx = data['tracesx']
tracesy = data['tracesy']
print (tracesx.shape)
print (tracesy.shape)


(89988, 4)
(89988, 4)


In [ ]:
tracesx_re = reassembled['tracesx_interpolated']
tracesy_re = reassembled['tracesy_interpolated']

# GET APPROXIMATE STATS FOR LABELED DATA

In [66]:
# CELL 5 : visualize results
animal_id = 0
t=np.arange(tracesx.shape[0])/25.

lens = []
for k in range(4):
    print (k)
    ax=plt.subplot(4,1,k+1)
    temp = tracesx[:,k]+tracesy[:,k]
    
    
    idx = np.isnan(temp)
    print (idx.shape)
    idx2 = np.where(idx==False)[0]
    plt.plot(t[idx2],temp[idx2],c='black')


    plt.plot(t,temp, c=clrs[k])

        
    #print (idx2.shape)
    #lens.append(idx2.shape[0])
    print (labels[k])
    plt.ylabel(labels[k]+ "\n" +str(np.round(idx2.shape[0]/tracesx.shape[0]*100, 0)) +"%",fontsize=14)
    
    plt.xlim(t[0],t[-1])
    if k!=3:
        plt.xticks([])
    else:
        plt.xlabel("Time (sec)",fontsize=20)
        
print (np.array(lens)/16572)
#for k in range(4):
#    ax=plt.subplot(4,1,k+1)
#    plt.plot(t,tracesy[:,k])

#plt.show()

0
(89988,)
female (red)
1
(89988,)
male (blue)
2
(89988,)
pup1 (cyan)
3
(89988,)
pup2 (green)
[]


In [61]:
# interpolate videos using LINEAR INTERPOLATION FOR ALL DATA
t=np.arange(tracesx.shape[0])/25.
from scipy import interpolate

tracesx_interp = tracesx.copy()
tracesy_interp = tracesy.copy()
for k in range(4):
    print (k)
    #ax=plt.subplot(4,1,k+1)
    current_val = None
    
    # determine if first frame has np.nan value; 
    # if so grab the first non np.nan val and backwards fill to the beginning.
    if np.isnan(tracesx_interp[0,k]):
        idx = np.where(np.isnan(tracesx_interp[:,k])==False)[0]
        tracesx_interp[:idx[0],k]=tracesx_interp[idx[0],k]
        tracesy_interp[:idx[0],k]=tracesy_interp[idx[0],k]
        
    # determine if last frame has np.nan value; 
    # if so grab the first non np.nan val and backwards fill to the beginning.
    if np.isnan(tracesx_interp[-1,k]):
        idx = np.where(np.isnan(tracesx_interp[:,k])==False)[0]
        tracesx_interp[idx[-1]:,k]=tracesx_interp[idx[-1],k]
        tracesy_interp[idx[-1]:,k]=tracesy_interp[idx[-1],k]
        #tracesy_interp[:idx[0],k]=tracesy_interp[idx[0],k]
        
    # second do linear interpolations across nans:
    # check where nans are:
    x = np.where(np.isnan(tracesx_interp[:,k])==False)[0]

    # request full x range data
    xnew = np.arange(0, tracesx_interp[:,k].shape[0], 1)

    # fill in missing data with linear interpolation
    f = interpolate.interp1d(x, tracesx_interp[x,k])

    tracesx_interp[:,k] = f(xnew)   # use interpolation function returned by `interp1d`
    
    # same with y vals
    y = np.where(np.isnan(tracesy_interp[:,1])==False)[0]
    f = interpolate.interp1d(x, tracesy_interp[x,k])
    tracesy_interp[:,k] = f(xnew)   # use interpolation function returned by `interp1d`

    #plt.plot(tracesx_interp[:,k])
    #plt.plot(tracesx_new+100)
    #break    
    
    
np.savez('/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressedvideo_labeled_exported_fixed_interpolated.npz',
        tracesx_interpolated = tracesx_interp,
        tracesy_interpolated = tracesy_interp,
        )   

#plt.show()      

0
1
2
3


In [8]:
# Add fixed labels to video:
reassembled = np.load('/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressedvideo_labeled_exported_corrected_interpolated_10timesteps.npz')
tracesx_interp = reassembled['tracesx_interpolated']
tracesy_interp = reassembled['tracesy_interpolated']

In [9]:
# CELL 5 : visualize results
animal_id = 0
t=np.arange(tracesx.shape[0])/25.

lens = []
for k in range(4):
    print (k)
    ax=plt.subplot(4,1,k+1)
    temp = tracesx_interp[:,k]+tracesy_interp[:,k]
    
    
    idx = np.isnan(temp)
    print (idx.shape)
    idx2 = np.where(idx==False)[0]
    plt.plot(t[idx2],temp[idx2],c='black')


    plt.plot(t,temp, c=clrs[k])

        
    #print (idx2.shape)
    #lens.append(idx2.shape[0])
    print (labels[k])
    plt.ylabel(labels[k]+ "\n" +str(np.round(idx2.shape[0]/tracesx.shape[0]*100, 0)) +"%",fontsize=14)
    
    plt.xlim(t[0],t[-1])
    if k!=3:
        plt.xticks([])
    else:
        plt.xlabel("Time (sec)",fontsize=20)
        
print (np.array(lens)/16572)
#for k in range(4):
#    ax=plt.subplot(4,1,k+1)
#    plt.plot(t,tracesy[:,k])


0
(89988,)
female (red)
1
(89988,)
male (blue)
2
(89988,)
pup1 (cyan)
3
(89988,)
pup2 (green)
[]


In [17]:


sums = np.zeros(tracesx_interp.shape[0], 'int32')
for k in range(4):
    idx = np.where(np.isnan(tracesx_interp[:,k])==False)[0]
    print (idx.shape)
    sums[idx]+=1
plt.plot(sums)
plt.show()

(76023,)
(63518,)
(54581,)
(43059,)


In [34]:
lengths = []
for k in range(4):
    lengths.append([])

start=0
n = sums[0]
for k in range(1,sums.shape[0]):
    if sums[k]==n:
        pass
    else:
        n= sums[k]
        lengths[n-1].append(k-start)
        start=k
        

In [42]:
for k in range(4):
    y = np.histogram(lengths[k], bins=np.arange(5,100,1))
    ax = plt.subplot(2,2,k+1)
    plt.plot(y[1][:-1]/25.,y[0])
    plt.title("# of animals: "+str(k+1))
    plt.xlabel("Seconds")
    plt.semilogy()
    plt.ylim(bottom=1)
plt.show()

In [62]:
# interpolate videos using LINEAR INTERPOLATION FOR ALL DATA
t=np.arange(tracesx.shape[0])/25.
from scipy import interpolate


#data = np.load('/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_01_54_23_358257_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npz')
data = np.load('/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressedvideo_labeled_exported_corrected.npz',allow_pickle=True)
tracesx = data['tracesx']
tracesy = data['tracesy']
print (tracesx.shape)
print (tracesy.shape)


tracesx_interp = tracesx.copy()
tracesy_interp = tracesy.copy()
max_interp=0
for k in range(4):
    print (k)
    #ax=plt.subplot(4,1,k+1)
    current_val = None
    
    # determine if first frame has np.nan value; 
    # if so grab the first non np.nan val and backwards fill to the beginning.
    if np.isnan(tracesx_interp[0,k]):
        idx = np.where(np.isnan(tracesx_interp[:,k])==False)[0]
        tracesx_interp[:idx[0],k]=tracesx_interp[idx[0],k]
        tracesy_interp[:idx[0],k]=tracesy_interp[idx[0],k]
        
    # determine if last frame has np.nan value; 
    # if so grab the first non np.nan val and backwards fill to the beginning.
    if np.isnan(tracesx_interp[-1,k]):
        idx = np.where(np.isnan(tracesx_interp[:,k])==False)[0]
        tracesx_interp[idx[-1]:,k]=tracesx_interp[idx[-1],k]
        tracesy_interp[idx[-1]:,k]=tracesy_interp[idx[-1],k]
        #tracesy_interp[:idx[0],k]=tracesy_interp[idx[0],k]
        
    # second do linear interpolations across nans:
    # check where nans are:
    for p in range(tracesx_interp[:,k].shape[0]-max_interp):
        # check if real val occurs in next 10 timesteups and replace val
        if np.isnan(tracesx_interp[p,k]):
            for l in range(1,max_interp,1):
                if np.isnan(tracesx_interp[p+l,k])==False:
                    tracesx_interp[p,k]=tracesx_interp[p+l,k]
                    tracesy_interp[p,k]=tracesy_interp[p+l,k]   # use interpolation function returned by `interp1d`

    #plt.plot(tracesx_interp[:,k])
    #plt.plot(tracesx_new+100)
    #break    
    
    
np.savez('/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressedvideo_labeled_exported_fixed_interpolated_10.npz',
        tracesx_interpolated = tracesx_interp,
        tracesy_interpolated = tracesy_interp,
        )   

#plt.show()      

(89988, 4)
(89988, 4)
0
1
2
3


# MAKE A VIDEO POST FIXING: 

In [64]:
# select start and ends:
# start = 51000
# end = start+5000

# Add fixed labels to video:
reassembled = np.load('/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressedvideo_labeled_exported_fixed_interpolated_10.npz')
tracesx_re = reassembled['tracesx_interpolated']
tracesy_re = reassembled['tracesy_interpolated']
start = 0
end = tracesx_re.shape[0]
#end = 1000
print (tracesx_re.shape)



# OPTIONAL MAKE VIDEO TO REVIEW ASSEMBLED VS. INFERENCE LABELS (PRE-FIX)
# colors have weird inversion; red is blue and cyan is yellow
colors_4 = ['blue','red','cyan','green']

video_name = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressed.avi'
original_vid = cv2.VideoCapture(video_name)

# video sizes
size_vid = np.array([1280,1024])
scale = 4
dot_size = 16//scale

#out_dir = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/'
fname_out = video_name[:-4]+"_corrected.mp4"
fourcc = cv2.VideoWriter_fourcc('M','P','E','G')
video_out = cv2.VideoWriter(fname_out,fourcc, 25, (size_vid[0]//scale,size_vid[1]//scale), True)

#dot_size = 10//scale
# setup cutoff 
pcutoff = 0.01

# go through first videos
from tqdm import trange

original_vid.set(cv2.CAP_PROP_POS_FRAMES, start)

font = cv2.FONT_HERSHEY_PLAIN

for n in trange(start, end, 1):
    ret, frame = original_vid.read()
    #print (n, frame.shape)
    cv2.putText(frame, str(n), (50, 100), font, 5, (255, 255, 0), 5)
    frame = frame[::scale, ::scale]
    
    #cv2.putText(frame,   text,location,  font,font size, font color,    font weight, line)
    #cv2.putText(img, text, pos, font_face, scale, color, 1, cv2.LINE_AA)
    
    for k in range(4):
        y = tracesx_re[n,k]
        x = tracesy_re[n,k]
        
        if np.isnan(x) or np.isnan(y):
            continue
        else:
            x=int(x)//scale
            y=int(y)//scale
            
            frame[x-dot_size:x+dot_size,y-dot_size:y+dot_size]= (np.float32(
                matplotlib.colors.to_rgb(colors_4[k]))*255.).astype('uint8')
            #print (colors_4[k])
            #frame[y-dot_size:y+dot_size,x-dot_size:x+dot_size]= (np.float32(
            #    matplotlib.colors.to_rgb(colors_4[z//14]))*255.).astype('uint8')
                
    #print ("")
    video_out.write(frame)

    #print ("")

video_out.release()
original_vid.release()
cv2.destroyAllWindows()


  0%|          | 34/89988 [00:00<04:26, 337.05it/s]

(89988, 4)


100%|██████████| 89988/89988 [03:43<00:00, 402.49it/s]


single click: button=1, x=20, y=31, xdata=0.531250, ydata=0.364583
0
2500
5000
7500
10000
12500
15000
17500
20000
22500
25000
27500
30000
32500
35000
37500
40000
42500
45000
47500
50000
52500
55000
57500
60000
62500
65000
67500
70000
72500
75000
77500
80000
82500
85000
87500


# CONVERT FROM SINGLE FEAUTRE BACK TO MULTIFEATURE

In [95]:
# convert single feature back to multi feature:
# Robust outlier detection function to crop some of the DLC errors;
def reject_outliers(data, m = 2.):
    d = np.abs(data - np.median(data))
    mdev = np.median(d)
    s = d/mdev if mdev else 0.
    return data[s<m]


# LOAD MULTI FEATURE DATA POST REASSEMBLY
csv_fname= '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_57_12_418305_compressed/2020-3-16_12_57_12_418305_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_bx_inference_fixed.npz'
traces = np.load(csv_fname)
tracesx_inference = traces['tracesx']
tracesy_inference = traces['tracesy']
likelihoods = traces['probs']


# LOAD SINGLE FEATURE POST GUI DATA:
# Add fixed labels to video:
reassembled = np.load('/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_57_12_418305_compressed/saved_traces_cat_reassemble.npz')
tracesx_re = reassembled['tracesx']
tracesy_re = reassembled['tracesy']
print ("traces reassbmeld: ", tracesy_re.shape)

print (tracesx_inference.shape)
print (tracesx_re.shape)
threshold = 0.01
start = 0
end = 89000
for k in range(start,end,1):    
    #if k%5000==0:
    #    print (k, "/", end)
        
    #tracesx_mean.append([])
    #tracesy_mean.append([])
    
    n_nans = 0
    xs = []
    ys = []
    # PLOT MEAN FEATURE
    for p in range(0, tracesx_inference.shape[0], 14):
        l = likelihoods[p:p+14,k]
        idx = np.where(l<threshold)[0]
        #print(idx.shape)

        traces_local = tracesx_inference[p:p+14,k]
        traces_local = np.delete(traces_local, idx)
        
        # remove nans from data
        idx_nan = np.where(np.isnan(traces_local)==True)[0]
        traces_local = np.delete(traces_local, idx_nan)

        # reject outliers
        traces_local = reject_outliers(traces_local, m = 4)

        x = np.nanmedian(traces_local)
        
        # find median of y also        
        traces_local = tracesy_inference[p:p+14,k]
        traces_local = np.delete(traces_local, idx)

        # remove nans from data
        idx_nan = np.where(np.isnan(traces_local)==True)[0]
        traces_local = np.delete(traces_local, idx_nan)

        # reject outliars
        traces_local = reject_outliers(traces_local, m = 4)
        y = np.nanmedian(traces_local)
        #print ("XY: ", x,y)
        
        xs.append(x)
        ys.append(y)
        if np.isnan(x) or np.isnan(y):
            
            # check to see if reassmbled animals also have nans
            if np.isnan(tracesx_re[k,p//14]) or np.isnan(tracesy_re[k,p//14]): 
                #print (k, p, x, y)
                n_nans+=1
    if n_nans>1:
        pre_nans = np.where(np.isnan(xs))[0].shape[0]
        post_nans = np.where(np.isnan(tracesx_re[k]))[0].shape[0]
        
        if pre_nans!=post_nans:
            print ("step :", k, "  has n_nasn: ", np.vstack((xs,ys)).T)
            print ("reasbmled: ", np.vstack((tracesx_re[k], tracesy_re[k])).T)
            print (" pre: ", pre_nans, "   post: ", post_nans)
            print ("")
print (tracesy_re[50001])
print (tracesx_re[50001])

traces reassbmeld:  (89989, 4)
(56, 89989)
(89989, 4)
step : 1371   has n_nasn:  [[293.361 502.241]
 [    nan     nan]
 [    nan     nan]
 [    nan     nan]]
reasbmled:  [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]]
 pre:  3    post:  4

step : 1372   has n_nasn:  [[243.259 278.624]
 [    nan     nan]
 [    nan     nan]
 [    nan     nan]]
reasbmled:  [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]]
 pre:  3    post:  4

step : 1373   has n_nasn:  [[243.259 278.624]
 [    nan     nan]
 [    nan     nan]
 [    nan     nan]]
reasbmled:  [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]]
 pre:  3    post:  4

step : 1374   has n_nasn:  [[243.259 278.624]
 [    nan     nan]
 [    nan     nan]
 [    nan     nan]]
reasbmled:  [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]]
 pre:  3    post:  4

step : 1375   has n_nasn:  [[283.823 495.634]
 [    nan     nan]
 [    nan     nan]
 [    nan     nan]]
reasbmled:  [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]]
 pre:  3    post:  4

step : 1376   has n_nasn:  [[28

step : 1552   has n_nasn:  [[251.515 612.466]
 [    nan     nan]
 [    nan     nan]
 [    nan     nan]]
reasbmled:  [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]]
 pre:  3    post:  4

step : 1553   has n_nasn:  [[258.75  612.557]
 [    nan     nan]
 [    nan     nan]
 [    nan     nan]]
reasbmled:  [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]]
 pre:  3    post:  4

step : 1554   has n_nasn:  [[251.838 612.457]
 [    nan     nan]
 [    nan     nan]
 [    nan     nan]]
reasbmled:  [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]]
 pre:  3    post:  4

step : 1684   has n_nasn:  [[260.0285 611.73  ]
 [     nan      nan]
 [     nan      nan]
 [     nan      nan]]
reasbmled:  [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]]
 pre:  3    post:  4

step : 1685   has n_nasn:  [[243.259 664.332]
 [    nan     nan]
 [    nan     nan]
 [    nan     nan]]
reasbmled:  [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]]
 pre:  3    post:  4

step : 1686   has n_nasn:  [[256.686 281.527]
 [    nan     nan]
 [    nan   

step : 7677   has n_nasn:  [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]]
reasbmled:  [[313.981 472.832]
 [    nan     nan]
 [    nan     nan]
 [    nan     nan]]
 pre:  4    post:  3

step : 7678   has n_nasn:  [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]]
reasbmled:  [[313.981 472.832]
 [    nan     nan]
 [    nan     nan]
 [    nan     nan]]
 pre:  4    post:  3

step : 7679   has n_nasn:  [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]]
reasbmled:  [[313.981 472.832]
 [    nan     nan]
 [    nan     nan]
 [    nan     nan]]
 pre:  4    post:  3

step : 7680   has n_nasn:  [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]]
reasbmled:  [[313.981 472.832]
 [    nan     nan]
 [    nan     nan]
 [    nan     nan]]
 pre:  4    post:  3

step : 7681   has n_nasn:  [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]]
reasbmled:  [[313.981 472.832]
 [    nan     nan]
 [    nan     nan]
 [    nan     nan]]
 pre:  4    post:  3

step : 7689   has n_nasn:  [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]]
reasbmled:  [

step : 8068   has n_nasn:  [[252.0505 628.526 ]
 [     nan      nan]
 [     nan      nan]
 [     nan      nan]]
reasbmled:  [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]]
 pre:  3    post:  4

step : 8069   has n_nasn:  [[243.424 632.174]
 [    nan     nan]
 [    nan     nan]
 [    nan     nan]]
reasbmled:  [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]]
 pre:  3    post:  4

step : 8070   has n_nasn:  [[243.83  632.095]
 [    nan     nan]
 [    nan     nan]
 [    nan     nan]]
reasbmled:  [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]]
 pre:  3    post:  4

step : 8105   has n_nasn:  [[251.9925 636.331 ]
 [     nan      nan]
 [     nan      nan]
 [     nan      nan]]
reasbmled:  [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]]
 pre:  3    post:  4

step : 8114   has n_nasn:  [[235.276 636.352]
 [    nan     nan]
 [    nan     nan]
 [    nan     nan]]
reasbmled:  [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]]
 pre:  3    post:  4

step : 8115   has n_nasn:  [[221.382 632.138]
 [    nan     nan]
 [  

step : 8418   has n_nasn:  [[     nan      nan]
 [313.7835 483.46  ]
 [     nan      nan]
 [     nan      nan]]
reasbmled:  [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]]
 pre:  3    post:  4

step : 8437   has n_nasn:  [[     nan      nan]
 [312.8855 483.1605]
 [     nan      nan]
 [     nan      nan]]
reasbmled:  [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]]
 pre:  3    post:  4

step : 8442   has n_nasn:  [[    nan     nan]
 [312.879 483.315]
 [    nan     nan]
 [    nan     nan]]
reasbmled:  [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]]
 pre:  3    post:  4

step : 8457   has n_nasn:  [[     nan      nan]
 [312.777  482.7245]
 [     nan      nan]
 [     nan      nan]]
reasbmled:  [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]]
 pre:  3    post:  4

step : 8458   has n_nasn:  [[     nan      nan]
 [312.6885 482.8085]
 [     nan      nan]
 [     nan      nan]]
reasbmled:  [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]]
 pre:  3    post:  4

step : 8459   has n_nasn:  [[     nan      nan]
 [313

...

step : 80041   has n_nasn:  [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]]
reasbmled:  [[     nan      nan]
 [     nan      nan]
 [     nan      nan]
 [344.3535 678.8075]]
 pre:  4    post:  3

step : 80042   has n_nasn:  [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]]
reasbmled:  [[     nan      nan]
 [     nan      nan]
 [     nan      nan]
 [344.3535 678.8075]]
 pre:  4    post:  3

step : 80043   has n_nasn:  [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]]
reasbmled:  [[     nan      nan]
 [     nan      nan]
 [     nan      nan]
 [344.3535 678.8075]]
 pre:  4    post:  3

step : 80044   has n_nasn:  [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]]
reasbmled:  [[     nan      nan]
 [     nan      nan]
 [     nan      nan]
 [344.3535 678.8075]]
 pre:  4    post:  3

step : 80045   has n_nasn:  [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]]
reasbmled:  [[     nan      nan]
 [     nan      nan]
 [     nan      nan]
 [344.3535 678.8075]]
 pre:  4    post:  3

step : 80046   has n_nasn:  [[nan nan]
 

step : 81862   has n_nasn:  [[267.275  683.0945]
 [     nan      nan]
 [     nan      nan]
 [     nan      nan]]
reasbmled:  [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]]
 pre:  3    post:  4

step : 81863   has n_nasn:  [[261.008  691.5945]
 [     nan      nan]
 [     nan      nan]
 [     nan      nan]]
reasbmled:  [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]]
 pre:  3    post:  4

step : 81864   has n_nasn:  [[243.259 278.624]
 [    nan     nan]
 [    nan     nan]
 [    nan     nan]]
reasbmled:  [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]]
 pre:  3    post:  4

step : 81865   has n_nasn:  [[243.259 278.624]
 [    nan     nan]
 [    nan     nan]
 [    nan     nan]]
reasbmled:  [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]]
 pre:  3    post:  4

step : 81866   has n_nasn:  [[243.259 278.624]
 [    nan     nan]
 [    nan     nan]
 [    nan     nan]]
reasbmled:  [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]]
 pre:  3    post:  4

step : 81867   has n_nasn:  [[243.259 278.624]
 [    nan     nan

step : 82600   has n_nasn:  [[253.748 678.727]
 [    nan     nan]
 [    nan     nan]
 [    nan     nan]]
reasbmled:  [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]]
 pre:  3    post:  4

step : 82623   has n_nasn:  [[263.1465 679.117 ]
 [     nan      nan]
 [     nan      nan]
 [     nan      nan]]
reasbmled:  [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]]
 pre:  3    post:  4

step : 82624   has n_nasn:  [[256.686 278.624]
 [    nan     nan]
 [    nan     nan]
 [    nan     nan]]
reasbmled:  [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]]
 pre:  3    post:  4

step : 82625   has n_nasn:  [[256.686 278.624]
 [    nan     nan]
 [    nan     nan]
 [    nan     nan]]
reasbmled:  [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]]
 pre:  3    post:  4

step : 82626   has n_nasn:  [[256.686 278.624]
 [    nan     nan]
 [    nan     nan]
 [    nan     nan]]
reasbmled:  [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]]
 pre:  3    post:  4

step : 82627   has n_nasn:  [[266.148  708.7735]
 [     nan      nan]
 [

step : 83123   has n_nasn:  [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]]
reasbmled:  [[    nan     nan]
 [    nan     nan]
 [    nan     nan]
 [298.615 689.025]]
 pre:  4    post:  3

step : 83124   has n_nasn:  [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]]
reasbmled:  [[    nan     nan]
 [    nan     nan]
 [    nan     nan]
 [298.615 689.025]]
 pre:  4    post:  3

step : 83126   has n_nasn:  [[309.912 677.38 ]
 [    nan     nan]
 [    nan     nan]
 [253.48  638.595]]
reasbmled:  [[    nan     nan]
 [    nan     nan]
 [    nan     nan]
 [298.615 689.025]]
 pre:  2    post:  3

step : 83128   has n_nasn:  [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]]
reasbmled:  [[    nan     nan]
 [    nan     nan]
 [    nan     nan]
 [298.615 689.025]]
 pre:  4    post:  3

step : 83129   has n_nasn:  [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]]
reasbmled:  [[    nan     nan]
 [    nan     nan]
 [    nan     nan]
 [298.615 689.025]]
 pre:  4    post:  3

step : 83130   has n_nasn:  [[nan nan]
 [nan nan

[193.3045 421.645  164.821  651.2065]
[1112.749   211.098   828.1045  687.36  ]


In [ ]:
# FUTURE WORK OBJECT ORIENTED PROGRAMMING

# reload data
#tracesx = tracesx_ip.copy() #data['tracesx_mean'].copy()
#tracesy = tracesy_ip.copy() #data['tracesy_mean'].copy()

# class dlc_object():

#     def __init__(self, tracesx, tracesy, traces_saved):
#         self.tracesx=tracesx.copy()
#         self.tracesy=tracesy.copy()        
#         self.traces_saved=traces_saved
        
#         self.current_index = []
#         self.selected_animal = []
#         self.original_vid = []
#         self.n_frames = []

        
        
# dlc = dlc_object(tracesx, tracesy,
#                 traces_saved)
# print (dlc.tracesx.shape)


In [2]:
# id_=2
# for k in range(tracesx.shape[0]):
#     if np.isnan(tracesx[k,id_])!=np.isnan(tracesy[k,id_]):
#         print (k, tracesx[k,id_], tracesy[k,id_])

NameError: name 'tracesx' is not defined